# **Coletando dados: SiSU UFPE**

---
Dados retirados de: https://www.ufpe.br/formas-de-ingresso/sisu-ufpe 


*   Anos: 2016, 2017, 2018 e 2019
*   Listagem dos candidatos classificados na 1ª chamada
*   A UFPE adotou o SiSU desde o vestibular do ano 2015, porém as cotas só foram inseridas na listagem divulgada para o público em 2016, por isso os dados escolhidos foram a partir de 2016

###Como são as modalidades de cotas na UFPE?

É válido destacar que qualquer modalidade de cota necessariamente o candidato deve ter cursado integralmente o ensino médio em escolas públicas.

Têm direito a cotas candidatos:
*   com renda familiar bruta per capita igual ou inferior a 1,5 salário mínimo
*   autodeclarados pretos, pardos ou indígenas
*   com deficiência

As modalidades são combinações desses pontos.



##Como os dados foram coletados

---

*   Os listões da UFPE são disponibilizados apenas em formato PDF

Exemplo de listão:

![listao](https://res.cloudinary.com/dgs7vlgfb/image/upload/v1554841888/Captura_de_Tela_2019-04-09_a%CC%80s_17.25.46.png)


Passo a passo:

1.   Foi utilizada uma [ferramenta](https://www.pdfmerge.com/pt/) para juntar os PDFs de todos os anos em um só
2.   Tendo um arquivo com todos os PDFs, foi utilizada outra [ferramenta](https://www.zamzar.com/) para a conversão para CSV
3.    Não foi possível utilizar a função direta do pandas para criação do dataframe, como é possível ver abaixo. Seria necessário olhar linha por linha do CSV para retirada do "lixo" e identificação de curso e ano, portanto foi escolhido criar o dataframe enquanto a leitura estava sendo feita



In [0]:
#get file
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [0]:
import csv, re
import pandas as pd

In [0]:
dfTeste = pd.read_csv('Sisu16-19.csv', header = None)
dfTeste.head()

,0,1,2,3,4,5,6,7,8,9
0,UFPE,-,SISU,-,SISTEMA DE SELEÇÃO UNIFICADA,2016,NaN,PAG 1,NaN,NaN
1,RELAÇÃO DOS CANDIDATOS CLASSIFICADOS NA 1A. CH...,NaN,NaN,NaN,NaN,NaN,NaN,18/01/2016,NaN,NaN
2,==============================================...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CURSO:,NaN,ABI - ENGENHARIA,NaN,GRAU:,ÁREA BÁSICA DE INGRESSO,TURNO:,INTEGRAL,CAMPUS:,RECIFE
4,INSC. ENEM,NaN,NaN,NaN,NOME DO CANDIDATO,IDENTIDADE,NOTA,MODALIDADE,NaN,NaN


## Criação do DataFrame

---
Dados presentes nos listões:

1.   Inscrição do ENEM
2.   Nome do candidato
3.   Identidade
4.   Nota
5.   Modalidade da cota
6.   Curso
7.   Ano

O código da *inscrição do ENEM* não entrará no DF por ser um dado irrelevante para o estudo.

A *identidade* também ficará de fora, para que haja o anonimato.

O *nome* ficará temporariamente até que seja feita a predição do gênero.

Dessa forma, o DF terá inicialmente as colunas:

1.   Nome
2.   Nota
3.   Modalidade
4.   Curso
5.   Ano





In [0]:
colNames = ['NOME', 'NOTA', 'MODALIDADE', 'CURSO', 'ANO']
df = pd.DataFrame(columns=colNames)
index = 0

In [0]:
regexCurso = r"(CURSO:)"
regexAno = r"(SELEÇÃO UNIFICADA)"
regexIgnorar = r"[a-zA-Z]|(=.*)|(\*.*)"

with open('Sisu16-19.csv', 'r') as f:
  reader = csv.reader(f)
  for row in reader:
    if(re.search(regexCurso, row[0]) != None):
      curso = row[2]
    if(re.search(regexAno, row[4]) != None):
      ano = row[5]
    if(re.search(regexIgnorar, row[0]) == None):
      df.loc[index] = [row[4], row[6], row[7], curso, ano]
      index += 1

In [0]:
df.head()

,NOME,NOTA,MODALIDADE,CURSO,ANO
0,ABINADI STANISCIA RUFINO DA SILVA,685.37,L4,ABI - ENGENHARIA,2016
1,ADALBERTO JOSE GUERRA PESSOA NETO,716.03,AC,ABI - ENGENHARIA,2016
2,ADRIANA SIQUEIRA CARNEIRO LEAO,659.49,L4,ABI - ENGENHARIA,2016
3,ADRIANO RODRIGO DA SILVA,648.32,L4,ABI - ENGENHARIA,2016
4,ADRIEL FILIPE LINS ALVES DA SILVA,695.95,AC,ABI - ENGENHARIA,2016


In [0]:
# salvar csv

from google.colab import files
df.to_csv("SiSU.csv")
files.download("SiSU.csv")